In [15]:
# Another workbook since the other one was slowing down.
# load some prereq libraries
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TKAgg')
from matplotlib import pyplot as plt
from sklearn import cross_validation

In [27]:
from stop_words import get_stop_words
import collections
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import word2vec
from lda.lda import LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import zero_one_loss
from pybrain.structure import FeedForwardNetwork
from pybrain.structure import RecurrentNetwork
from sklearn.svm import SVC
from pybrain.structure import LinearLayer, SigmoidLayer
from sklearn.metrics.pairwise import chi2_kernel
from pybrain.structure import FullConnection
from pybrain.datasets import ClassificationDataSet
from pybrain.utilities           import percentError
from pybrain.tools.shortcuts     import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure.modules   import SoftmaxLayer
import nltk
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import confusion_matrix

In [2]:
############################### Library Functions ###########################################

In [3]:
def process_text(e):
    raw = e.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in en_stop]
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    return texts

In [4]:
def process_text_no_stem(e):
    raw = e.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    return tokens

In [5]:
def label_to_num(ys):
    y_true = []
    for l in ys:
        if l == 'Positive':
            y_true.append(0)
        if l == 'Neutral':
            y_true.append(1)
        if l == 'Negative':
            y_true.append(2)
    return y_true

In [6]:
def create_token_vector(tokens):
    v = np.zeros(len(vocabulary_set))
    for t in tokens:
        v[vocabulary_set.index(t)] = 1
    return v

In [7]:
def create_feature_vector(tokens, candidate, location, subject):
    # returns the feature vector that represents this
    v = np.zeros(len(vocabulary_set) + 3)
    for t in tokens:
        v[vocabulary_set.index(t)] = 1
    # get candidate id
    v[len(vocabulary_set)] = candidate_set.index(candidate)
    v[len(vocabulary_set) + 1] = location_set.index(location)
    v[len(vocabulary_set) + 2] = subject_set.index(subject)
    return v

In [8]:
def create_feature_vector_expanded(tokens, candidate, location, subject):
    # returns the feature vector that represents this
    v = np.zeros(len(vocabulary_set) + len(candidate_set) + len(subject_set) + len(location_set))
    for t in tokens:
        v[vocabulary_set.index(t)] = 1

    # get candidate id
    v[len(vocabulary_set) + candidate_set.index(candidate)] = 1
    v[len(vocabulary_set) + len(candidate_set) + location_set.index(location)] = 1
    v[len(vocabulary_set) + len(candidate_set) + len(location_set) + subject_set.index(subject)] = 1
    return v

In [9]:
def create_feature_vector_cls(candidate, location, subject):
    v = np.zeros(len(candidate_set) + len(subject_set) + len(location_set))
    
    # get candidate id
    v[candidate_set.index(candidate)] = 1
    v[len(candidate_set) + location_set.index(location)] = 1
    v[len(candidate_set) + len(location_set) + subject_set.index(subject)] = 1
    return v

In [10]:
def create_feature_vector_pos(tokens):
    v = np.zeros(len(pos_set))
    tags = nltk.pos_tag(tokens)
    for w, t in tags:
        v[pos_sl.index(t)] = v[pos_sl.index(t)] + 1
    
    return v

In [36]:
def create_location_vector(l):
    v = np.zeros(len(location_set))
    v[location_set.index(l)] = 1
    return v

In [37]:
def create_subject_vector(sm):
    v = np.zeros(len(subject_set))
    v[subject_set.index(sm)] = 1
    return v

In [32]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [33]:
labels = ["Positive", "Neutral", "Negative"]

In [11]:
############################## End Library Functions #########################################

In [22]:
pos_set = set()
for s in dftrain['text']:
#     print s
    raw = s.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    tags = nltk.pos_tag(tokens)
#     print tags
    for w, t in tags:
        pos_set.add(t)
print pos_set

set(['PRP$', 'VBG', 'VBD', 'VBN', 'VBP', 'WDT', 'JJ', 'WP', 'VBZ', 'DT', 'RP', '$', 'NN', 'FW', 'TO', 'PRP', 'RB', 'NNS', 'NNP', 'VB', 'WRB', 'CC', 'PDT', 'RBS', 'RBR', 'CD', 'EX', 'IN', 'WP$', 'MD', 'NNPS', 'JJS', 'JJR', 'SYM', 'UH'])


In [23]:
pos_sl = list(pos_set)

In [16]:
tokenizer = RegexpTokenizer(r'\w+')
p_stemmer = PorterStemmer()

en_stop = get_stop_words('en')

In [17]:
dftrain = pd.read_csv('output/Sentiment.csv')

In [18]:
df = dftrain['text']
texts_all = []
for e in df:
    raw = e.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if i not in en_stop]
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    texts_all.append(texts)

In [19]:
vocabulary = [item for sublist in texts_all for item in sublist]
# print vocabulary
vocabulary_set = list(set(vocabulary))

In [20]:
candidate_list = []
for c in dftrain['candidate']:
    candidate_list.append(c)
candidate_set = list(set(candidate_list))

subject_list = []
for s in dftrain['subject_matter']:
    subject_list.append(s)
subject_set = list(set(subject_list))

location_list = []
for l in dftrain['tweet_location']:
    location_list.append(l)
location_set = list(set(location_list))

In [21]:
# get a partial set
X_train, X_test, y_train, y_test = cross_validation.train_test_split(dftrain['text'], dftrain['sentiment'],
                                                                    test_size=0.4, random_state=0)

In [25]:
# build an LDA model using this X_train set
m = np.array([np.zeros(len(vocabulary_set)) for i in range(len(X_train))])
for i in range(len(X_train)):
    tokens = process_text(X_train.iloc[i])
    for t in tokens:
        m[i][vocabulary_set.index(t)] = 1

In [32]:
# we have a matrix, lets use LDA on it.
model = LDA(n_topics=10, random_state=0)
model.fit(m.astype(int))

<lda.lda.LDA instance at 0x10b700b00>

In [55]:
model.doc_topic_.shape

(8322, 10)

In [41]:
# we have the LDA vectors, which represent the probability of each topic. Lets feed this in as a matrix to a classifier
# see how the classifier performs given the dimension reduction that has happened.
clf = DecisionTreeClassifier(random_state=0)
clf.fit(model.doc_topic_, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [46]:
skf = StratifiedKFold(dftrain['sentiment'],n_folds=10)
averageError = 0.0

In [34]:
k = 10
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
avg_cm = np.zeros((3, 3))
for train_index, test_index in skf:
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    m = np.array([np.zeros(len(vocabulary_set)) for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text(X_train.iloc[i])
        for t in tokens:
            m[i][vocabulary_set.index(t)] = 1
    
    model = LDA(n_topics=13, random_state=0)
    model.fit(m.astype(int))
    
    m_test = np.array([np.zeros(len(vocabulary_set)) for i in range(len(X_test))])
    for i in range(len(X_test)):
        tokens = process_text(X_test.iloc[i])
        for t in tokens:
            m_test[i][vocabulary_set.index(t)] = 1
            
    clf_X_test = model.transform(m_test.astype(int))
    
    clf = RandomForestClassifier(random_state=0)
    clf.fit(model.doc_topic_, y_train)
    y_pred = clf.predict(clf_X_test)
    error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
    print error
    avg_cm += confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(avg_cm / 1.0, title="Confusion Matrix - RF + LDA")
    plt.show()
    break
    averageError += (1./k) * error
print "Average error: %4.2f%s" % (100 * averageError,'%')
plot_confusion_matrix(avg_cm / k, title="Confusion Matrix - RF + LDA")
plt.show()

0.434125269978
Average error: 0.00%


In [61]:
print len(dftrain['sentiment'])

13871


In [63]:
print len(dftrain['text'])

13871


In [96]:
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
for train_index, test_index in skf:
    print dftrain['subject_matter'][train_index[0]]

None of the above
None of the above
None of the above
None of the above
None of the above
None of the above
None of the above
None of the above
None of the above
None of the above


In [40]:
k = 10
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
avg_cm = np.zeros((3,3))
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    z_array = np.zeros(len(pos_set) + len(location_set) + len(subject_set))
    m = np.array([z_array for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text_no_stem(X_train.iloc[i])
#         candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
        v = np.concatenate((create_feature_vector_pos(tokens), create_location_vector(tweet_location)))
        m[i] = np.concatenate((v, create_subject_vector(sm)))
    
#     model = LDA(n_topics=5, random_state=0)
#     model.fit(m.astype(int))
    
    z_array = np.zeros(len(pos_set) + len(location_set) + len(subject_set))
    m_test = np.array([z_array for i in range(len(X_test))])
    for i in range(len(X_test)):
        tokens = process_text_no_stem(X_test.iloc[i])
#         candidate = dftrain['candidate'][test_index[i]]
        tweet_location = dftrain['tweet_location'][test_index[i]]
        sm = dftrain['subject_matter'][test_index[i]]
        v = np.concatenate((create_feature_vector_pos(tokens), create_location_vector(tweet_location)))
        m_test[i] = np.concatenate((v, create_subject_vector(sm)))
            
#     clf_X_test = model.transform(m_test.astype(int))
    
    clf = RandomForestClassifier(random_state=0)
    clf.fit(m, y_train)
    y_pred = clf.predict(m_test)
    error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
    print error
    averageError += (1./k) * error
    avg_cm += confusion_matrix(y_test, y_pred)
print "Average error: %4.2f%s" % (100 * averageError,'%')
plot_confusion_matrix(avg_cm / k)
plt.show()

0.392368610511
0.389488840893
0.39409221902
0.419610670512
0.416005767844
0.426820475847
0.352092352092
0.391053391053
0.348484848485
0.367965367965
Average error: 38.98%


In [114]:
k = 10
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    m = np.array([np.zeros(len(vocabulary_set) + 3) for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text(X_train.iloc[i])
        candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
        m[i] = create_feature_vector(tokens, candidate, tweet_location, sm)
    print m
#     model = LDA(n_topics=13, random_state=0)
#     model.fit(m.astype(int))
    
    m_test = np.array([np.zeros(len(vocabulary_set) + 3) for i in range(len(X_test))])
    for i in range(len(X_test)):
        tokens = process_text(X_test.iloc[i])
        candidate = dftrain['candidate'][test_index[i]]
        tweet_location = dftrain['tweet_location'][test_index[i]]
        sm = dftrain['subject_matter'][test_index[i]]
        m_test[i] = create_feature_vector(tokens, candidate, tweet_location, sm)
    print m_test
#     clf_X_test = model.transform(m_test.astype(int))
    
    clf = RandomForestClassifier(random_state=0)
    clf.fit(m, y_train)
    y_pred = clf.predict(m_test)
    error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
    print error
    averageError += (1./k) * error
print "Average error: %4.2f%s" % (100 * averageError,'%')

[[    0.     0.     0. ...,     8.  1962.    12.]
 [    0.     0.     0. ...,    11.  2078.    12.]
 [    0.     0.     0. ...,     8.  3307.    12.]
 ..., 
 [    0.     0.     0. ...,    11.     0.    12.]
 [    0.     0.     0. ...,     5.     0.    11.]
 [    0.     0.     0. ...,    11.  2201.    12.]]
[[    0.     0.     0. ...,     8.     0.    12.]
 [    0.     0.     0. ...,     4.     0.    12.]
 [    0.     0.     0. ...,     8.     0.    12.]
 ..., 
 [    0.     0.     0. ...,     8.  1670.    12.]
 [    0.     0.     0. ...,     5.  2691.    12.]
 [    0.     0.     0. ...,     8.     0.    11.]]
0.347732181425
[[    0.     0.     0. ...,     8.     0.    12.]
 [    0.     0.     0. ...,     4.     0.    12.]
 [    0.     0.     0. ...,     8.     0.    12.]
 ..., 
 [    0.     0.     0. ...,    11.     0.    12.]
 [    0.     0.     0. ...,     5.     0.    11.]
 [    0.     0.     0. ...,    11.  2201.    12.]]
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   8

In [ ]:
k = 10
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    m = np.array([np.zeros(len(vocabulary_set) + 3) for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text(X_train.iloc[i])
        candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
        m[i] = create_feature_vector(tokens, candidate, tweet_location, sm)
    print m
#     model = LDA(n_topics=13, random_state=0)
#     model.fit(m.astype(int))
    
    m_test = np.array([np.zeros(len(vocabulary_set) + 3) for i in range(len(X_test))])
    for i in range(len(X_test)):
        tokens = process_text(X_test.iloc[i])
        candidate = dftrain['candidate'][test_index[i]]
        tweet_location = dftrain['tweet_location'][test_index[i]]
        sm = dftrain['subject_matter'][test_index[i]]
        m_test[i] = create_feature_vector(tokens, candidate, tweet_location, sm)
    print m_test
#     clf_X_test = model.transform(m_test.astype(int))
    
    clf = SVC(kernel=chi2_kernel)
    clf.fit(m, y_train)
    y_pred = clf.predict(m_test)
    error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
    print error
    averageError += (1./k) * error
print "Average error: %4.2f%s" % (100 * averageError,'%')

In [24]:
## building a neural network system to test with here as well.
def createNetwork(inputDim, hiddenDim, outputDim):
    n = FeedForwardNetwork()
    n.addInputModule(LinearLayer(inputDim, name='in'))
    n.addModule(SigmoidLayer(hiddenDim, name='hidden'))
    n.addModule(SigmoidLayer(hiddenDim, name='hidden1'))
    n.addModule(SigmoidLayer(hiddenDim, name='hidden2'))
    n.addOutputModule(SoftmaxLayer(outputDim, name='out'))
    n.addConnection(FullConnection(n['in'], n['hidden'], name='c1'))
    n.addConnection(FullConnection(n['hidden'], n['hidden1'], name='c2'))
    n.addConnection(FullConnection(n['hidden1'], n['hidden2'], name='c3'))
    n.addConnection(FullConnection(n['hidden2'], n['out'], name='c4'))
    n.sortModules()
    return n

In [50]:
k = 100
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    input_dim = 300
    trndata = ClassificationDataSet(input_dim, nb_classes=3)
    z_array = np.zeros(len(vocabulary_set) + len(candidate_set) + len(location_set) + len(subject_set) + len(pos_set))
#     z_array = np.zeros(len(pos_set))
    m = np.array([z_array for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text_no_stem(X_train.iloc[i])
        tokens_stemmed = process_text(X_train.iloc[i])
        candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
        l = label_to_num([y_train.iloc[i]])[0]
        v1 = create_feature_vector_expanded(tokens_stemmed, candidate, tweet_location, sm)
        v2 = create_feature_vector_pos(tokens)
#         trndata.addSample(np.concatenate((v1,v2)), [l])
        m[i] = np.concatenate((v1, v2))
#         m[i] = v2
    
    # feature select on this m
    selectkb = SelectKBest(f_classif, k=input_dim)
    m_new = selectkb.fit_transform(m, y_train)
    
    for i in range(len(m_new)):
        l = label_to_num([y_train.iloc[i]])[0]
        trndata.addSample(m_new[i], [l])
    
    print trndata    
    
    testdata = ClassificationDataSet(input_dim, nb_classes=3)
    z_array = np.zeros(len(vocabulary_set) + len(candidate_set) + len(location_set) + len(subject_set) + len(pos_set))
#     z_array = np.zeros(len(pos_set))
    m_test = np.array([z_array for i in range(len(X_test))])
    for i in range(len(X_test)):
        tokens = process_text_no_stem(X_test.iloc[i])
        tokens_stemmed = process_text(X_test.iloc[i])
        candidate = dftrain['candidate'][test_index[i]]
        tweet_location = dftrain['tweet_location'][test_index[i]]
        sm = dftrain['subject_matter'][test_index[i]]
        l = label_to_num([y_test.iloc[i]])[0]
        v1 = create_feature_vector_expanded(tokens_stemmed, candidate, tweet_location, sm)
        v2 = create_feature_vector_pos(tokens)
#         testdata.addSample(np.concatenate((v1,v2)), [l])
        m_test[i] = np.concatenate((v1, v2))
#         m_test[i] = v2
    
    # now transform and add to testdata
    m_test_new = selectkb.transform(m_test)
    for i in range(len(m_test)):
        l = label_to_num([y_test.iloc[i]])[0]
        testdata.addSample(m_test_new[i], [l])
    
    print testdata
    # convert to one of many
    trndata._convertToOneOfMany()
    testdata._convertToOneOfMany()
    # train the network
    n = createNetwork(trndata.indim, 200, trndata.outdim)
    print n
    trainer = BackpropTrainer(n, dataset=trndata, momentum=0.1, verbose=True, weightdecay=0.0001)
    
    # train for a certain amount of epochs
    epoch_list = []
    test_error_list = []
    train_error_list = []
    for i in range(50):
        print "Training Epoch: {0}".format(i)
        trainer.trainEpochs(1)
        trnresult = percentError( trainer.testOnClassData(),
                              trndata['class'] )
        tstresult = percentError( trainer.testOnClassData(
               dataset=testdata ), testdata['class'] )
        epoch_list.append(i)
        test_error_list.append(tstresult)
        train_error_list.append(trnresult)
        print "epoch: %4d" % trainer.totalepochs, \
              "  train error: %5.2f%%" % trnresult, \
              "  test error: %5.2f%%" % tstresult
    error = percentError(trainer.testOnClassData(dataset=testdata), testdata['class'])
    averageError += (1./k) * error
    out = n.activateOnDataset(testdata)
    out = out.argmax(axis=1)
    cm = confusion_matrix(label_to_num(y_test), out)
    plot_confusion_matrix(cm, title="Confusion Matrix: NN with F-Test")
    plt.show()
    plt.figure()
    plt.plot(epoch_list, test_error_list, color="red")
    plt.plot(epoch_list, train_error_list, color="green")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.title("Error as a Function of Epochs Trained")
    plt.show()
    break
print "Average error: %4.2f%s" % (100 * averageError,'%')

input: dim(16382, 300)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  1.  2.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  2.  0.]
 [ 0.  0.  0. ...,  1.  1.  0.]]

target: dim(16382, 1)
[[0]
 [0]
 [0]
 ..., 
 [0]
 [2]
 [0]]

class: dim(0, 1)
[]


input: dim(254, 300)
[[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  1.  2.  1.]
 [ 0.  0.  0. ...,  0.  2.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  1.  0.]
 [ 0.  0.  0. ...,  0.  4.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]

target: dim(254, 1)
[[1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [2]
 [1]
 [2]
 [2]
 [2]
 [0]
 [2]
 [1]
 [0]
 [2]
 [2]
 [2]
 [2]
 [2]
 [0]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [0]
 [2]
 [1]
 [0]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [0]
 [2]
 [2]
 [0]
 [0]
 [2]
 [1]
 [1]
 [1]
 [0]
 [2]
 [2]
 [2]
 [2]
 [0]
 [2]
 [0]
 [2]
 [2]
 [2]
 [1]
 [2]
 [2]
 [0]
 [0]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 

In [35]:
# POS_TAG RF classifier
k = 3
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
avg_cm = np.zeros((3,3))
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    z_array = np.zeros(len(vocabulary_set) + len(candidate_set) + len(location_set) + len(subject_set) + len(pos_set))
    m = np.array([z_array for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text_no_stem(X_train.iloc[i])
        tokens_stemmed = process_text(X_train.iloc[i])
        candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
        v1 = create_feature_vector_expanded(tokens_stemmed, candidate, tweet_location, sm)
        m[i] = np.concatenate((create_feature_vector_pos(tokens), v1))
    print m
#     model = LDA(n_topics=13, random_state=0)
#     model.fit(m.astype(int))

    selectkb = SelectKBest(f_classif, k=150)
    m_new = selectkb.fit_transform(m, y_train)
    
    z_array = np.zeros(len(vocabulary_set) + len(candidate_set) + len(location_set) + len(subject_set) + len(pos_set))
    m_test = np.array([z_array for i in range(len(X_test))])
    for i in range(len(X_test)):
        tokens = process_text_no_stem(X_test.iloc[i])
        tokens_stemmed = process_text(X_test.iloc[i])
        candidate = dftrain['candidate'][test_index[i]]
        tweet_location = dftrain['tweet_location'][test_index[i]]
        sm = dftrain['subject_matter'][test_index[i]]
        v1 = create_feature_vector_expanded(tokens_stemmed, candidate, tweet_location, sm)
        m_test[i] = np.concatenate((create_feature_vector_pos(tokens), v1))
    print m_test
#     clf_X_test = model.transform(m_test.astype(int))

    m_test_new = selectkb.transform(m_test)
    
    clf = RandomForestClassifier(random_state=0)
    clf.fit(m, y_train)
    y_pred = clf.predict(m_test)
    error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
    print error
    avg_cm += confusion_matrix(y_test, y_pred)
    averageError += (1./k) * error
print "Average error: %4.2f%s" % (100 * averageError,'%')
plot_confusion_matrix(avg_cm / k)
plt.show()

[[ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  1.  1. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  2.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


/Users/abkhanna/Documents/workspace/Princeton/cos424/a2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [   33    35    36 ..., 19670 19678 19679] are constant.
  UserWarning)


[[ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
0.372108108108
[[ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  2.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


/Users/abkhanna/Documents/workspace/Princeton/cos424/a2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [   38    47    49 ..., 19682 19685 19689] are constant.
  UserWarning)


[[ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  1.  1. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
0.425048669695
[[ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


/Users/abkhanna/Documents/workspace/Princeton/cos424/a2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [   40    55    75 ..., 19683 19687 19688] are constant.
  UserWarning)


[[ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  1.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  2.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
0.343067272334
Average error: 38.01%


In [72]:
k = 10
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    z_array = np.zeros(len(candidate_set) + len(location_set) + len(subject_set) + len(pos_set))
    m = np.array([z_array for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text_no_stem(X_train.iloc[i])
        candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
        v1 = create_feature_vector_cls(candidate, tweet_location, sm)
        v2 = create_feature_vector_pos(tokens)
        m[i] = np.concatenate((v1, v2))
    
    model = LDA(n_topics=15, random_state=0)
    model.fit(m.astype(int))
    
    z_array = np.zeros(len(candidate_set) + len(location_set) + len(subject_set) + len(pos_set))
    m_test = np.array([z_array for i in range(len(X_test))])
    for i in range(len(X_test)):
        tokens = process_text_no_stem(X_test.iloc[i])
        candidate = dftrain['candidate'][test_index[i]]
        tweet_location = dftrain['tweet_location'][test_index[i]]
        sm = dftrain['subject_matter'][test_index[i]]
        v1 = create_feature_vector_cls(candidate, tweet_location, sm)
        v2 = create_feature_vector_pos(tokens)
        m_test[i] = np.concatenate((v1, v2))
            
    clf_X_test = model.transform(m_test.astype(int))
    
    clf = RandomForestClassifier(random_state=0)
    clf.fit(model.doc_topic_, y_train)
    y_pred = clf.predict(clf_X_test)
    error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
    print error
    averageError += (1./k) * error
print "Average error: %4.2f%s" % (100 * averageError,'%')

0.544276457883


KeyboardInterrupt: 

In [82]:
# Feature selection time
# Feature selection time
# POS_TAG RF classifier
k = 10
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    z_array = np.zeros(len(candidate_set) + len(location_set) + len(subject_set) + len(pos_set))
    m = np.array([z_array for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text_no_stem(X_train.iloc[i])
        candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
        v1 = create_feature_vector_cls(candidate, tweet_location, sm)
        m[i] = np.concatenate((create_feature_vector_pos(tokens), v1))
    print m
#     model = LDA(n_topics=13, random_state=0)
#     model.fit(m.astype(int))

    skbest = SelectKBest(f_classif, k=100)
    skbest.fit(m, y_train)
    print skbest.scores_
    print describe(skbest.scores_, nan_policy='omit')
    
#     clf = RandomForestClassifier(random_state=0)
#     clf.fit(m, y_train)
#     y_pred = clf.predict(m_test)
#     error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
#     print error
#     averageError += (1./k) * error
# print "Average error: %4.2f%s" % (100 * averageError,'%')

[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  2.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
[  15.34051944   60.12570275   41.50017614 ...,   11.39547514  106.37947709
  281.65913004]
DescribeResult(nobs=array(3997), minmax=(masked_array(data = 0.047994131649,
             mask = False,
       fill_value = 1e+20)
, masked_array(data = 359.513441906,
             mask = False,
       fill_value = 1e+20)
), mean=3.2387702681422308, variance=209.28695978159089, skewness=masked_array(data = 16.6895913072,
             mask = False,
       fill_value = 1e+20)
, kurtosis=316.27154602208469)
[[ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  1.]
 [ 0.  2.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
[  16.60920519   64.41464555   42.59031343 ...,   10.31995046  113.46744237
  265.10517266]
Describe

KeyboardInterrupt: 

In [28]:
# Feature selection time
# Feature selection time
# POS_TAG RF classifier
k = 10
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    z_array = np.zeros(len(vocabulary_set) + len(candidate_set) + len(location_set) + len(subject_set) + len(pos_set))
    m = np.array([z_array for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text_no_stem(X_train.iloc[i])
        tokens_stemmed = process_text(X_train.iloc[i])
        candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
        v1 = create_feature_vector_expanded(tokens_stemmed, candidate, tweet_location, sm)
        m[i] = np.concatenate((create_feature_vector_pos(tokens), v1))
    print m
#     model = LDA(n_topics=13, random_state=0)
#     model.fit(m.astype(int))

    F, p_val = f_classif(m, y_train)
    f = open("f-scores.txt", "w")
    f.write("Length of POS: {0}, Length of Vocab: {1}, Length of Candidate: {2}, Length of location: {3}".format(
            len(pos_set), len(vocabulary_set), len(candidate_set), len(location_set)))
    for i in range(len(F)):
        f.write("Score: {0}, Column: {1}\n".format(F[i], i))
    f.close()
    break
    
#     clf = RandomForestClassifier(random_state=0)
#     clf.fit(m, y_train)
#     y_pred = clf.predict(m_test)
#     error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
#     print error
#     averageError += (1./k) * error
# print "Average error: %4.2f%s" % (100 * averageError,'%')
print "done"

KeyboardInterrupt: 